In [1]:
import requests
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd 

In [2]:
def embajadas():
    response = requests.get('https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json')
    results = response.json()
    embajadas = pd.json_normalize(results['@graph'])
    embajadas["Dirección"] = embajadas["address.street-address"]
    return embajadas.to_json("../data/embajadas.json", orient='records')
def estaciones():
    estaciones = pd.read_json("../data/bicimad2.json", orient='records')
    geometry_coordinates = estaciones["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace(",","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    estaciones = pd.concat([stations, geometry_coordinates], axis=1)
    return estaciones.to_json("../data/estaciones.json", orient='records')


In [3]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)
    


In [4]:
embajadas = pd.read_json('../data/embajadas.json')
estaciones = pd.read_json('../data/estaciones.json')

In [5]:
response = requests.get('https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json')
results = response.json()
pd.json_normalize(results(['@graph']))

TypeError: 'dict' object is not callable

In [6]:
embajadas["mercator_start"] = embajadas.apply(lambda x: to_mercator(x["LATITUD"], x["LONGITUD"]), axis=1)
estaciones["mercator_finish"] = estaciones.apply(lambda x: to_mercator(x["LATITUD"], x["LONGITUD"]), axis=1)


In [7]:
estaciones_clean = estaciones[["name","address","LONGITUD","LATITUD", "mercator_finish"]]
estaciones_clean = estaciones_clean.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Longitud_final", "LATITUD": "Latitud_final"})


In [8]:
embajadas_clean = embajadas[["NOMBRE","Dirección","LONGITUD","LATITUD", "mercator_start"]]
embajadas_clean = embajadas_clean.rename(columns={"NOMBRE": "Place of interest", "Dirección": "Place address", "LONGITUD": "Longitud_inicial", "LATITUD": "Latitud_inicial"}, errors="raise")


In [9]:
df_resultado = pd.merge(embajadas_clean,estaciones_clean, how='cross')

In [10]:
df_resultado["Distancia"] = df_resultado.apply(lambda x: distance_meters(x["mercator_start"], x["mercator_finish"]), axis=1)
df_resultado


,Place of interest,Place address,Longitud_inicial,Latitud_inicial,mercator_start,BiciMAD station,Station location,Longitud_final,Latitud_final,mercator_finish,Distancia
0,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Puerta del Sol A,Puerta del Sol nº 1,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837),1595.648926
1,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Puerta del Sol B,Puerta del Sol nº 1,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782),1570.712528
2,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Miguel Moya,Calle Miguel Moya nº 1,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317),1697.402416
3,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148262),1606.478855
4,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Malasaña,Calle Manuela Malasaña nº 5,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012),1107.400364
...,...,...,...,...,...,...,...,...,...,...,...
42763,Embajada del Nepal,CALLE GENERAL ORAA Nº3,-3.687905,40.436031,POINT (4501318.352334742 -410819.4374695785),Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,-3.726990,40.443750,POINT (4502177.655770458 -415179.5220676512),4443.955456
42764,Embajada del Nepal,CALLE GENERAL ORAA Nº3,-3.687905,40.436031,POINT (4501318.352334742 -410819.4374695785),Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,-3.726930,40.443420,POINT (4502140.920338497 -415172.8287427756),4430.421391
42765,Embajada del Nepal,CALLE GENERAL ORAA Nº3,-3.687905,40.436031,POINT (4501318.352334742 -410819.4374695785),Facultad Biología,Calle José Antonio Novais frente al nº 12,-3.727295,40.448332,POINT (4502687.743941171 -415213.4906984274),4602.492493
42766,Embajada del Nepal,CALLE GENERAL ORAA Nº3,-3.687905,40.436031,POINT (4501318.352334742 -410819.4374695785),Facultad Derecho,Avenida Complutense nº 23,-3.729370,40.451090,POINT (4502994.74083288 -415445.024322674),4919.993103


In [76]:
def minimum():
    df_minimo = df_resultado[df_resultado["Place of interest"] == input('Pon el lugar de interés: ')]
    df_minimum = df_minimo[df_minimo['Distancia'] == df_minimo['Distancia'].min()]
    minimum_mod = df_minimum[["Place of interest","Place address","BiciMAD station","Station location"]]
    return minimum_mod

In [77]:
minimum()

Pon el lugar de interés: Embajada del Nepal


,Place of interest,Place address,BiciMAD station,Station location
42663,Embajada del Nepal,CALLE GENERAL ORAA Nº3,Paseo de la Castellana - Glorieta de Emilio Ca...,Paseo de la Castellana nº 43


In [124]:
def all_min():
    return df_resultado.sort_values(by = "Distancia", ascending = True).groupby('Place of interest')['Place address', 'Distancia','BiciMAD station', 'Station location'].nth(0).drop(["Distancia"], axis = "columns")

In [125]:
all_min()

/Users/Andrea/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Place address,BiciMAD station,Station location
Place of interest,,,
Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,Alonso Martínez,Plaza de Alonso Martínez nº 5
Consulado de Belice,CALLE TALAVERA Nº9,Serrano 210,Calle Serrano nº 210
Consulado de Bolivia,CALLE AVIADOR LINDBERGH Nº3,María Francisca 1,Calle María Francisca nº 1
Consulado de Brasil,CALLE GOYA Nº5,Plaza de Colón,Calle Goya nº 1
Consulado de Bélgica,PASEO CASTELLANA Nº18,Castellana,Paseo de la Castellana nº 4
...,...,...,...
Embajada de Vietnam,AVENIDA ALFONSO XIII Nº54,Corazón de María,Calle Santa Hortensia nº 31
Embajada de Yemen,PASEO CASTELLANA Nº117,Sor Ángela de la Cruz,Calle Sor Ángela de la Cruz nº 2
Embajada de la República Popular Democrática de Corea,CALLE DARIO APARICIO Nº43,Facultad Derecho,Avenida Complutense nº 23


In [108]:
def all_minimum():
    df_minimum = pd.DataFrame(columns=["Place of interest","Type of place","Place address","BiciMAD station","Station location"])
    for place in monuments_clean["Place of interest"]:
        df_filter2 = df_sumary[df_sumary["Place of interest"] == place]
        df_places =  df_filter2[df_filter2['Distance'] == df_filter2['Distance'].min()]
        df_places_clear = df_places[["Place of interest","Type of place","Place address","BiciMAD station","Station location"]]
        df_minimum = df_minimum.append(df_places_clear)
    return df_minimum

In [109]:
def minimum():
    i = input('Pon un lugar de interes: ')
    df_minimo = df_resultado[df_resultado["Place of interest"] == i]
    a = df_minimo.sort_values(by="Distancia", ascending=True).head(1)
    minimum = pre_minimum[[“Place of interest”,“Type of place”,“Place address”,“BiciMAD station”,“Station location”]]
    return minimum
    return a


SyntaxError: invalid character in identifier (2698173674.py, line 5)